# KISS-ICP running on the Newer College Dataset

The whole purpose of this notebook is to have a reproducable entry point for the experiments of the paper. You can also modify the system and just run this notebook to inspect the overall results.

We use the "old" version of the newer college dataset, the one that uses the OS1-64 LiDAR. There are 2 public sequences, the "long_experiment" is only avaiable as rosbag file.

## Expected dataset layout

```sh
newer_college
  ├── 01_short_experiment
  │   ├── ground_truth
  │   │   ├── poses_kitti_format.txt
  │   │   └── registered_poses.csv
  │   ├── raw_format
  │   │   ├── ouster_imu
  │   │   ├── ouster_scan
  │   │   │   ├── cloud_1583836591_182590976.pcd
  │   │   │   ├── cloud_1583836591_282592512.pcd
  │   │   │   ├── ...
  │   │   └── realsense_imu
  │   └── time_offsets
  │       └── time_offsets.csv
  ├── 02_long_experiment
  │   ├── ground_truth
  │   │   └── registered_poses.csv
  │   ├── raw_format
  │   │   └── ouster_scan
  │   └── rosbag
  │       ├── rooster_2020-03-10-11-36-51_0.bag
  │       ├── rooster_2020-03-10-11-39-38_1.bag
  │       ├── ...

```

In [ ]:
# Install KISS-ICP and Plotting tools
%pip install kiss-icp ipympl evo >/dev/null

import os
from pathlib import Path

import kiss_icp
import matplotlib.pyplot as plt
import numpy as np
from evo.tools import plot
from kiss_icp.datasets import dataset_factory
from kiss_icp.pipeline import OdometryPipeline

%autoreload 2
%matplotlib inline
%matplotlib widget

In [ ]:
data_root = os.environ.get("DATASETS")
data_dir = Path(os.path.join(data_root, "newer_college/2020-ouster-os1-64-realsense"))
print(f"Reading datasets from : {data_root}")

## Custom Dataloader for the `02_long_experiment` sequence

For whatever reason this part of the dataset is only public as rosbag. So we need to do a bit of extra effort to run the experiment (but not too much ;)

It is also a nice exercise on how can you extend KISS-ICP to process data from your own dataset by simply providing a custom dataloader (just one python class!)

In [ ]:
import numpy as np
import rosbag
import sensor_msgs.point_cloud2 as pc2
from natsort import natsorted
from pyquaternion import Quaternion


class NewerCollegeRosbag:
    def __init__(self, data_dir: Path):
        self.data_source = os.path.join(data_dir, "")
        self.pose_file = os.path.join(self.data_source, "ground_truth/registered_poses.csv")
        self.gt_poses = self.load_gt_poses(self.pose_file)
        self.sequence_id = "02_long_experiment"

        # process rosbag-files
        print("Processing rosbag files (this will take some time)")
        self.topic = "/os1_cloud_node/points"
        self.rosbag_dir = data_dir / "rosbag"
        self.rosbag_files = natsorted(
            [
                os.path.join(self.rosbag_dir, f)
                for f in os.listdir(self.rosbag_dir)
                if f.endswith(".bag")
            ]
        )
        self.bags = []
        self.n_scans = 0
        for rosbag_file in self.rosbag_files:
            bag = rosbag.Bag(rosbag_file, mode="r")
            self.bags.append(bag)
            self.n_scans += bag.get_message_count(topic_filters=self.topic)

        self.bag = self.bags.pop(0)
        self.msgs = self.bag.read_messages(topics=[self.topic])
        print(f"Now processing {self.bag.filename}")

    def __len__(self):
        return self.n_scans

    def __getitem__(self, idx):
        try:
            _, msg, _ = next(self.msgs)
        except StopIteration:
            self.bag.close()
            # new bagile
            self.bag = self.bags.pop(0)
            self.msgs = self.bag.read_messages(topics=[self.topic])
            print(f"Now processing {self.bag.filename}")
            _, msg, _ = next(self.msgs)

        points = np.array(list(pc2.read_points(msg, field_names=["x", "y", "z"])))
        return points.astype(np.float64)

    @staticmethod
    def load_gt_poses(file_path: str):
        """Taken from pyLiDAR-SLAM/blob/master/slam/dataset/nhcd_dataset.py"""
        ground_truth_df = np.genfromtxt(str(file_path), delimiter=",", dtype=np.float64)
        xyz = ground_truth_df[:, 2:5]
        rotations = np.array(
            [
                Quaternion(x=x, y=y, z=z, w=w).rotation_matrix
                for x, y, z, w in ground_truth_df[:, 5:]
            ]
        )

        num_poses = rotations.shape[0]
        poses = np.eye(4, dtype=np.float64).reshape(1, 4, 4).repeat(num_poses, axis=0)
        poses[:, :3, :3] = rotations
        poses[:, :3, 3] = xyz

        T_CL = np.eye(4, dtype=np.float32)
        T_CL[:3, :3] = Quaternion(x=0, y=0, z=0.924, w=0.383).rotation_matrix
        T_CL[:3, 3] = np.array([-0.084, -0.025, 0.050], dtype=np.float32)
        poses = np.einsum("nij,jk->nik", poses, T_CL)
        poses = np.einsum("ij,njk->nik", np.linalg.inv(poses[0]), poses)
        return poses

In [ ]:
from kiss_icp_eval import run_sequence


def ncd_short_sequence(int):
    return OdometryPipeline(
        dataset=dataset_factory(
            dataloader="ncd",
            data_dir=data_dir / "01_short_experiment",
        ),
    )


def ncd_long_sequence(int):
    return OdometryPipeline(
        dataset=NewerCollegeRosbag(data_dir=data_dir / "02_long_experiment"),
    )


results = {}
run_sequence(ncd_short_sequence, sequence="01_short_experiment", results=results)
run_sequence(ncd_long_sequence, sequence="02_long_experiment", results=results)

In [ ]:
from kiss_icp_eval import print_metrics_table

print_metrics_table(results)

In [ ]:
from kiss_icp_eval import plot_trajectories

plot_trajectories(results)